In [1]:
from binance.client import Client
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
# binance trial key
api_key = '3x1g9GeEGc8QFwRPbXPtmhpL5G6dFIhOsf4P8TG3mptnNXptIhIdytYAGU9PH9KH'
api_secret = 'ApKGT7B7gMyOTrAeUEXKl5WtSMCd1hm4Qy5ngVcwean1zXJryCbaXNe6heoInhU1'
client = Client(api_key,api_secret)

ModuleNotFoundError: No module named 'requests'

In [116]:
start_date = "1 Jan, 2020"
end_date = "31 Dec, 2020"
interval = Client.KLINE_INTERVAL_5MINUTE #1DAY
# for more available interval: https://python-binance.readthedocs.io/en/latest/constants.html
col = ['open_time','open','high','low','close','volume','close_time','asset_volume',
       'num_trades','taker_base_volume','taker_quote_volume','symbol']
symbols = ['BTCUSDT','ETHUSDT','UNIUSDT','SOLUSDT','APEUSDT','MANAUSDT']
# symbols = [i.get('symbol') for i in client.get_all_tickers()]

In [117]:
%%time
df = pd.DataFrame()
for sym in symbols:
    print(f"running klines for {sym}")
    klines = client.get_historical_klines(sym,interval,start_date,end_date)
    temp = pd.DataFrame(klines).iloc[:,:-1]
    temp = temp.astype(float)
    temp['symbol'] = sym
    df = df.append(temp)

running klines for BTCUSDT
running klines for ETHUSDT
running klines for UNIUSDT
running klines for SOLUSDT
running klines for APEUSDT
running klines for MANAUSDT
CPU times: user 5.54 s, sys: 619 ms, total: 6.16 s
Wall time: 2min 31s


In [118]:
# clean raw df
df.columns = col
df['open_time'] = df['open_time'].apply(lambda x: datetime.fromtimestamp(x/1000.0)) #get rid of millisecond
df['close_time'] = df['close_time'].apply(lambda x: datetime.fromtimestamp(x/1000.0))
df['pct_chg'] = df['close'].pct_change()*100
df.drop(columns=['close_time','volume','taker_base_volume'],inplace=True) #drop base valued (ie in BTC)
df.reset_index(drop=True,inplace=True)
print(df.shape)
df.head()

(322781, 10)


,open_time,open,high,low,close,asset_volume,num_trades,taker_quote_volume,symbol,pct_chg
0,2020-01-01 08:00:00,7195.24,7196.25,7178.64,7179.78,686317.136252,1127.0,235537.295045,BTCUSDT,NaN
1,2020-01-01 08:05:00,7179.76,7191.77,7178.20,7191.07,426481.260364,631.0,177935.618201,BTCUSDT,0.157247
2,2020-01-01 08:10:00,7193.15,7193.53,7180.24,7180.97,345446.503019,694.0,139596.621683,BTCUSDT,-0.140452
3,2020-01-01 08:15:00,7180.97,7186.40,7177.35,7178.29,231162.555424,576.0,93091.433276,BTCUSDT,-0.037321
4,2020-01-01 08:20:00,7177.71,7182.46,7175.47,7176.96,351927.893881,710.0,163817.881155,BTCUSDT,-0.018528


##### Volume
- asset_volume = Volume traded, in quote asset (USDT)

##### Taker base/quote volume
- Taker buy base asset volume represents how many of the total base asset volume are contributed by the taker buy orders. Once taker_buy_base_asset_volume and the total volume (Volume) is known, the value of the following cases ( taker_sell_volume, maker_buy_volume, maker_sell_volume ) are clear.Every trade has a buyer and a seller. A buyer can be a maker or a taker. But when a buyer is a maker, the seller must be a taker, and vice versa. 
- taker_buy_base_asset_volume = maker_sell_base_asset_volume
- taker_sell_base_asset_volume = maker_buy_base_asset_volume
- total_volume = taker_buy_base_asset_volume + taker_sell_base_asset_volume = maker_buy_base_asset_volume + maker_sell_base_asset_volume

In [119]:
df.to_feather('binance_kline_2020.feather')

In [127]:
df_2020 = pd.read_feather('binance_kline_2020.feather')
df_2021 = pd.read_feather('binance_kline_2021.feather')
df_2022 = pd.read_feather('binance_kline_2022.feather')

In [134]:
all_df = pd.concat([df_2020,df_2021,df_2022])
all_df.ffill(inplace=True)
all_df.reset_index(drop=True,inplace=True)
print(all_df.shape)
all_df.head()

(1136961, 10)


,open_time,open,high,low,close,asset_volume,num_trades,taker_quote_volume,symbol,pct_chg
0,2020-01-01 08:00:00,7195.24,7196.25,7178.64,7179.78,686317.136252,1127.0,235537.295045,BTCUSDT,NaN
1,2020-01-01 08:05:00,7179.76,7191.77,7178.20,7191.07,426481.260364,631.0,177935.618201,BTCUSDT,0.157247
2,2020-01-01 08:10:00,7193.15,7193.53,7180.24,7180.97,345446.503019,694.0,139596.621683,BTCUSDT,-0.140452
3,2020-01-01 08:15:00,7180.97,7186.40,7177.35,7178.29,231162.555424,576.0,93091.433276,BTCUSDT,-0.037321
4,2020-01-01 08:20:00,7177.71,7182.46,7175.47,7176.96,351927.893881,710.0,163817.881155,BTCUSDT,-0.018528


In [135]:
all_df.to_feather('binance_kline.feather')

### Manage dataset

In [2]:
import pandas as pd
df = pd.read_feather('binance_kline.feather')

In [3]:
df.head()

,open_time,open,high,low,close,asset_volume,num_trades,taker_quote_volume,symbol,pct_chg
0,2020-01-01 08:00:00,7195.24,7196.25,7178.64,7179.78,686317.136252,1127.0,235537.295045,BTCUSDT,NaN
1,2020-01-01 08:05:00,7179.76,7191.77,7178.20,7191.07,426481.260364,631.0,177935.618201,BTCUSDT,0.157247
2,2020-01-01 08:10:00,7193.15,7193.53,7180.24,7180.97,345446.503019,694.0,139596.621683,BTCUSDT,-0.140452
3,2020-01-01 08:15:00,7180.97,7186.40,7177.35,7178.29,231162.555424,576.0,93091.433276,BTCUSDT,-0.037321
4,2020-01-01 08:20:00,7177.71,7182.46,7175.47,7176.96,351927.893881,710.0,163817.881155,BTCUSDT,-0.018528


In [4]:
sym = list(df['symbol'].unique())
sym

['BTCUSDT', 'ETHUSDT', 'UNIUSDT', 'SOLUSDT', 'MANAUSDT', 'APEUSDT']

In [5]:
for s in sym:
    sym_df = df[df['symbol'] == s]
    df_name = f"{s}.csv"
    sym_df.to_csv(df_name)